In [1]:
import sys

import torch
import numpy as np

import attacks
from basemodels import TorchModelContainer, IrisNN, BCNN
from datasets import DATASET_LIST, DataContainer, get_dataset_list
# from defences import DefenceContainer

%load_ext autoreload
%autoreload 2

In [2]:
print(sys.version)
print(*sys.path, sep='\n')

3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]
/usr/lib/python36.zip
/usr/lib/python3.6
/usr/lib/python3.6/lib-dynload

/home/lukec/venv/lib/python3.6/site-packages
/home/lukec/.local/lib/python3.6/site-packages
/usr/local/lib/python3.6/dist-packages
/usr/lib/python3/dist-packages
/home/lukec/venv/lib/python3.6/site-packages/IPython/extensions
/home/lukec/.ipython


In [3]:
print('Avaliable datasets:')
print(get_dataset_list())

Avaliable datasets:
['MNIST', 'CIFAR10', 'SVHN', 'BankNote', 'BreastCancerWisconsin', 'HTRU2', 'Iris', 'WheatSeed']


In [4]:
DATA_ROOT = 'data'
BATCH_SIZE = 128

# image datasets: {'MNIST', 'CIFAR10', 'SVHN'}
# quantitative datasets: {'BankNote', 'BreastCancerWisconsin', 'HTRU2', 'Iris', 'WheatSeed'}
NAME = 'BreastCancerWisconsin'
print(f'Starting {NAME} data container...')
print(DATASET_LIST[NAME])

dc = DataContainer(DATASET_LIST[NAME], DATA_ROOT)
dc(size_train=0.8, normalize=True)

num_features = dc.dim_data[0]
num_classes = dc.num_classes
print('Features:', num_features)
print('Classes:', num_classes)

## model in {BCNN, IrisNN, MnistCnnCW}
# model = IrisNN(num_features, num_classes, hidden_nodes=16)  # for Iris
model = BCNN(num_features, num_classes)
model_name = model.__class__.__name__
print('Using model:', model_name)

mc = TorchModelContainer(model, dc)
mc.fit(epochs=100, batch_size=BATCH_SIZE)

Starting BreastCancerWisconsin data container...
{'name': 'BreastCancerWisconsin', 'type': 'quantitative', 'size': 569, 'num_classes': 2, 'dim_data': (30,)}
Loading data...
Preparing DataFrame...
Reading from data/BreastCancerWisconsin.csv
Spliting train/test sets into numpy arrays...
Successfully load data! Time taken:  0m 0.0s
Features: 30
Classes: 2
Using model: BCNN
Using device: cuda:0
{'lr': 0.0001, 'betas': (0.9, 0.999)}
[ 1/100]  0m 0.0s - Train Loss: 0.6807 Acc: 61.0989% - Test Loss: 0.6711 Acc: 69.2982%
[ 2/100]  0m 0.0s - Train Loss: 0.6791 Acc: 61.0989% - Test Loss: 0.6695 Acc: 69.2982%
[ 3/100]  0m 0.0s - Train Loss: 0.6777 Acc: 61.0989% - Test Loss: 0.6680 Acc: 69.2982%
[ 4/100]  0m 0.0s - Train Loss: 0.6764 Acc: 61.0989% - Test Loss: 0.6664 Acc: 69.2982%
[ 5/100]  0m 0.0s - Train Loss: 0.6751 Acc: 61.0989% - Test Loss: 0.6649 Acc: 69.2982%
[ 6/100]  0m 0.0s - Train Loss: 0.6737 Acc: 61.0989% - Test Loss: 0.6634 Acc: 69.2982%
[ 7/100]  0m 0.0s - Train Loss: 0.6723 Acc: 61

In [5]:
dc.data_test_np.shape

(114, 30)

In [6]:
attack = attacks.CarliniL2Container(mc)
print(attack.attack_params)

{'confidence': 0.0, 'targeted': False, 'learning_rate': 0.01, 'binary_search_steps': 10, 'max_iter': 100, 'initial_const': 0.01, 'max_halving': 5, 'max_doubling': 10, 'batch_size': 8}


In [7]:
n = 100
# targets = attack.randam_targets(n, dc.num_classes)
# adv, y_adv, x_clean, y_clean = attack.generate(count=n, targets=targets)
adv, y_adv, x_clean, y_clean = attack.generate(count=n)
print(attack.attack_params)
print(adv.shape)
print(y_adv.shape)
print(x_clean.shape)
print(y_clean.shape)

Time taken for training 100 adversarial examples:  1m 17.2s
{'confidence': 0.0, 'targeted': False, 'learning_rate': 0.01, 'binary_search_steps': 10, 'max_iter': 100, 'initial_const': 0.01, 'max_halving': 5, 'max_doubling': 10, 'batch_size': 8}
(100, 30)
(100,)
(100, 30)
(100,)


In [8]:
accuracy = mc.evaluate(x_clean, y_clean)
print(f'Accuracy on clean samples: {accuracy*100:.4f}%')
accuracy = mc.evaluate(adv, y_clean)
print(f'Accuracy on adversarial example: {accuracy*100:.4f}%')

Accuracy on clean samples: 100.0000%
Accuracy on adversarial example: 29.0000%
